In [1]:
pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in c:\programdata\anaconda3\lib\site-packages (1.12.8)Note: you may need to restart the kernel to use updated packages.



In [43]:
from google.oauth2 import service_account
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload
from googleapiclient.discovery import build
import pprint
import os
import httplib2 
import requests
from oauth2client.service_account import ServiceAccountCredentials
import urllib.request
import zipfile
from requests.exceptions import HTTPError

pp = pprint.PrettyPrinter(indent=4)

In [30]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'task2-download.json'
SERVICE_ACCOUNT_FILE_w = 'task2-write.json'
spreadsheet_id = '1A6XaXqjBvh78RvLkCjSu8C6MNlxMcGlU67L6R_8tpDI'

In [31]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
httpAuth = credentials.authorize(httplib2.Http())
service = build('sheets', 'v4', http = httpAuth)

credentials_w = ServiceAccountCredentials.from_json_keyfile_name(
        SERVICE_ACCOUNT_FILE_w, scopes=SCOPES)
httpAuth_w = credentials_w.authorize(httplib2.Http())
service_w = build('sheets', 'v4', http = httpAuth_w)

In [9]:
spreadsheet = service.spreadsheets().get(spreadsheetId = spreadsheet_id).execute()
sheetList = spreadsheet.get('sheets')
sheet_name, sheet_id = [], []
for sheet in sheetList:
    sheet_name.append(sheet['properties']['title'])
    sheet_id.append(sheet['properties']['sheetId'])

In [10]:
def get_values(ranges, spreadsheet_id):   
    results = service.spreadsheets().values().batchGet(spreadsheetId = spreadsheet_id, 
                                     ranges = ranges, 
                                     valueRenderOption = 'FORMATTED_VALUE',  
                                     dateTimeRenderOption = 'FORMATTED_STRING').execute() 
    return results['valueRanges'][0]['values']

In [34]:
path_folder_for_zip = 'C://Users/strelokask/Desktop/Amina/_project2/files'
if os.path.exists(path_folder_for_zip):
    print("Указанный файл существует") 
else:
    os.mkdir(path_folder_for_zip)

In [12]:
folder_start_column = 'A'
folder_start_row = '2'
url_start_column = 'B'
url_start_row = '2'
max_number_of_urls = '7'
max_folder_count = '700'

url_end_ = chr(ord(url_start_column) + int(max_number_of_urls))
folder_start = folder_start_column + folder_start_row
url_start = url_start_column + url_start_row
range_ = chr(ord(folder_start_column) + int(max_number_of_urls)) + max_folder_count

print(url_end_)
print(folder_start)
print(url_start)
print(range_)

I
A2
B2
H700


In [56]:
folders = []
for folder_name in sheet_name[1:]:  
    if os.path.exists(path_folder_for_zip + '/' + folder_name):
        print("Указанный файл существует") 
    else:
        os.mkdir(path_folder_for_zip + '/' + folder_name)
#    count_img = []    
    path_of_directory_root = path_folder_for_zip + '/' + folder_name + '/'
    ranges = [folder_name + '!' + folder_start + ':' + range_]  
    sheet_values = get_values(ranges, spreadsheet_id)
    for elem in sheet_values[65:]:  
        path_of_directory = path_of_directory_root + elem[0]
        folders.append(elem[0])
        if os.path.exists(path_of_directory):
            print("Указанный файл существует") 
        else:
            os.mkdir(path_of_directory)
        count = len(elem)
        for i in range(1, count):
            url = 'http://' + elem[i]
            try:
                response = requests.get(url)
                response.raise_for_status()
            except HTTPError as http_err:
                print(f'HTTP error occurred: {http_err}')  # Python 3.6
            except Exception as err:
                print(f'Other error occurred: {err}')  # Python 3.6
            else:
                path_for_image = path_of_directory + '/' + str(i) + '.jpg'
                with open(path_for_image, "wb") as f:
                    f.write(response.content)

'''        count_img.append(len(elem) - 1)
    print('ya tut')
    range_sheet = folder_name + '!' + url_end_ + url_start_row + ':' + url_end_ + max_folder_count
    
    print(range_sheet)
    values = service_w.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheet_id,
                                                       body = {'valueInputOption':'USER_ENTERED',
                                                               'data' : [{'range': range_sheet,
                                                                          'majorDimension':'ROWS',
                                                                          'values': count_img
                                                                          }]
                                                              }
                                                      ).execute()'''

Указанный файл существует
Указанный файл существует
Other error occurred: Invalid URL 'http://': No host supplied


"        count_img.append(len(elem) - 1)\n    print('ya tut')\n    range_sheet = folder_name + '!' + url_end_ + url_start_row + ':' + url_end_ + max_folder_count\n    \n    print(range_sheet)\n    values = service_w.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheet_id,\n                                                       body = {'valueInputOption':'USER_ENTERED',\n                                                               'data' : [{'range': range_sheet,\n                                                                          'majorDimension':'ROWS',\n                                                                          'values': count_img\n                                                                          }]\n                                                              }\n                                                      ).execute()"